In [13]:
import scipy
import matplotlib.pyplot as plt
import os
import ffmpeg
import moviepy.editor as mp
from scipy.io.wavfile import read
import numpy as np
from scipy.fft import fft, ifft
import math
import numpy as np
from pathlib import Path
#from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from typing import Tuple, Dict, Union
import ffmpeg

In [14]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [15]:
video_waltter_path = os.path.join(DATA_FOLDER, "example_waltter.MOV")
video_vikture_path = os.path.join(DATA_FOLDER, "example_vikture.MOV")

video_waltter_e15_path = os.path.join(DATA_FOLDER, "example_waltter_early_15s.MOV")
video_vikture_e15_path = os.path.join(DATA_FOLDER, "example_vikture_early_15s.MOV")

video_waltter_l15_path = os.path.join(DATA_FOLDER, "example_waltter_late_15s.MOV")
video_vikture_l15_path = os.path.join(DATA_FOLDER, "example_vikture_late_15s.MOV")

video_waltter_e30_path = os.path.join(DATA_FOLDER, "example_waltter_early_30s.MOV")
video_vikture_e30_path = os.path.join(DATA_FOLDER, "example_vikture_early_30s.MOV")

video_keparoicam_L = os.path.join(DATA_FOLDER, "keparoicam_clipL.mp4")
video_keparoicam_R = os.path.join(DATA_FOLDER, "keparoicam_clipR.mp4")

video_keparoicam_example_L = os.path.join(DATA_FOLDER, "example_videos_left", "keparoicam_left_part1.mp4")
video_keparoicam_example_R = os.path.join(DATA_FOLDER, "example_videos_right", "keparoicam_right_part1.mp4")

In [16]:
# Get delay from synchronize_audio notebook
delay = 0.2263265306122449

In [17]:
import ast
import operator as op

allowed_operators = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
                     ast.Div: op.truediv, ast.Pow: op.pow, ast.BitXor: op.xor,
                     ast.USub: op.neg}

def eval_expr(expr):
    """
    >>> eval_expr('2^6')
    4
    >>> eval_expr('2**6')
    64
    >>> eval_expr('1 + 2*3**(4^5) / (6 + -7)')
    -5.0
    """
    return eval_(ast.parse(expr, mode='eval').body)

def eval_(node):
    if isinstance(node, ast.Num):  # <number>
        return node.n
    elif isinstance(node, ast.BinOp):  # <left> <operator> <right>
        return allowed_operators[type(node.op)](eval_(node.left), eval_(node.right))
    elif isinstance(node, ast.UnaryOp):  # <operator> <operand> e.g., -1
        return allowed_operators[type(node.op)](eval_(node.operand))
    else:
        raise TypeError(node)

def get_video_info(video_path) -> Dict[str, Union[int, float, str]]:
    probe = ffmpeg.probe(video_path)
    file_path = str(probe['format']['filename'])
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    duration = float(probe['format']['duration'])
    frame_rate = round(eval_expr(video_stream['avg_frame_rate']))
    # Frame rate removed in notebook because cannot import relative local utils
    
    return {
        "file_path" : file_path,
        "frame_height": height,
        "frame_width": width,
        "frame_rate": frame_rate,
        "duration": duration
    }

In [18]:
print(get_video_info(video_keparoicam_example_L))

{'file_path': '/home/jakki/git_personal/meow/ml/data/example_videos_left/keparoicam_left_part1.mp4', 'frame_height': 1080, 'frame_width': 1920, 'frame_rate': 60, 'duration': 30.016}


In [19]:
def extract_filename_without_extension(path) -> str:
    filename = Path(path).stem
    return filename


def split_file_from_extension(path, without_dot=True) -> Tuple[str, str]:
    filename, extension = os.path.splitext(path)
    if without_dot:
        extension = extension.replace('.', '').lower()
    return filename, extension

def extract_folder_from_path(path) -> str:
    return os.path.dirname(path)


# Separate path to three parts: folder, filename, file extension
def separate_path_to_parts(path) -> Tuple[str, str, str]:
    filename = extract_filename_without_extension(path)
    file_extension = split_file_from_extension(path)[1]
    folder = extract_folder_from_path(path)
    
    return folder, filename, file_extension


print(separate_path_to_parts(video_keparoicam_example_L))

('/home/jakki/git_personal/meow/ml/data/example_videos_left', 'keparoicam_left_part1', 'mp4')


In [20]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

In [21]:
# If delay is positive, audio1 needs to be delayed and negative if audio2 needs to be delayed
def synchronize_videos(video1_path, video2_path, delay: float, output_path=None):
    video1_info = get_video_info(video1_path)
    video2_info = get_video_info(video2_path)
    video1_path_separated = separate_path_to_parts(video1_path)
    video2_path_separated = separate_path_to_parts(video2_path)

    final_duration = min(video1_info['duration'], video2_info['duration']) - abs(delay)
    
    if delay >= 0:
        print(f"Delay {video1_path} by {delay} seconds")
        video1_start_time = delay
        video1_end_time = final_duration + delay
        video1_name = os.path.join(video1_path_separated[0], f"{video1_path_separated[1]}_synchronized.{video1_path_separated[2]}")
        video2_start_time = 0
        video2_end_time = final_duration
        video2_name = os.path.join(video2_path_separated[0], f"{video2_path_separated[1]}_synchronized.{video2_path_separated[2]}")

        ffmpeg_extract_subclip(video1_path, video1_start_time, video1_end_time, targetname=video1_name)
        ffmpeg_extract_subclip(video2_path, video2_start_time, video2_end_time, targetname=video2_name)

    else:
        print(f"Delay {video2_path} by {delay} seconds")
        video1_start_time = 0
        video1_end_time = final_duration
        video1_name = os.path.join(video1_path_separated[0], f"{video1_path_separated[1]}_synchronized.{video1_path_separated[2]}")

        video2_start_time = abs(delay)
        video2_end_time = final_duration + abs(delay)
        video2_name = os.path.join(video2_path_separated[0], f"{video2_path_separated[1]}_synchronized.{video2_path_separated[2]}")

        ffmpeg_extract_subclip(video1_path, video1_start_time, video1_end_time, targetname=video1_name)
        ffmpeg_extract_subclip(video2_path, video2_start_time, video2_end_time, targetname=video2_name)

In [22]:
synchronize_videos(video_keparoicam_example_L, video_keparoicam_example_R, delay=delay)

Delay /home/jakki/git_personal/meow/ml/data/example_videos_left/keparoicam_left_part1.mp4 by 0.2263265306122449 seconds
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
